In [5]:
!pip install google-cloud-storage google-cloud-aiplatform

  Using cached google_cloud_storage-3.0.0-py2.py3-none-any.whl.metadata (12 kB)
  Using cached google_cloud_aiplatform-1.80.0-py2.py3-none-any.whl.metadata (32 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached google_api_core-2.24.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_crc32c-1.6.0-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
  Using cached proto_plus-1.26.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached google_cloud_storage-2.19.0-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached google_cloud_bigquery-3.29.0-py2.py3-none-any.whl.metadata (7.6 kB)
  Using cached google_cloud_resource_manager-1.14.0-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached shapely-2.0.7-cp312-cp312-win_amd64.whl.metadata (7.1 kB)
  Using cached docstring_parser-0.16-py3-none-any.whl

# Create Bucket

In [27]:
import os

# Set the path to your service account key JSON file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "capstone-ml-451002-bef2906dcf92.json"

# Test authentication
from google.cloud import storage

client = storage.Client()
buckets = list(client.list_buckets())
print("Authentication successful. Buckets:", [b.name for b in buckets])


Authentication successful. Buckets: ['ml_ai_capstone_stock']


# Upload Packages

In [21]:
from google.cloud import storage
import os

bucket_name = "ml_ai_capstone_stock"
model_path = "ml_model/"  # The converted model directory
destination_blob_name = "models/ml_model/"  # Cloud Storage path

client = storage.Client()
bucket = client.bucket(bucket_name)

# Upload the entire SavedModel directory, including empty folders
for root, dirs, files in os.walk(model_path):
    # Ensure empty directories (like assets/ and variables/) are created in GCS
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        relative_dir_path = os.path.relpath(dir_path, model_path).replace("\\", "/") + "/"  # Convert to GCS-compatible path
        blob_path = os.path.join(destination_blob_name, relative_dir_path).replace("\\", "/")

        # Create an empty blob to represent the folder in GCS
        blob = bucket.blob(blob_path)
        blob.upload_from_string("")  # Upload an empty string to create the folder

        print(f"Created empty folder gs://{bucket_name}/{blob_path}")

    # Upload all files in the current directory
    for file in files:
        local_path = os.path.join(root, file)
        relative_path = os.path.relpath(local_path, model_path).replace("\\", "/")  # Normalize path for GCS
        blob_path = os.path.join(destination_blob_name, relative_path).replace("\\", "/")

        # Upload the file to GCS
        blob = bucket.blob(blob_path)
        blob.upload_from_filename(local_path)

        print(f"Uploaded {local_path} to gs://{bucket_name}/{blob_path}")

Created empty folder gs://ml_ai_capstone_stock/models/ml_model/assets/
Created empty folder gs://ml_ai_capstone_stock/models/ml_model/variables/
Uploaded ml_model/fingerprint.pb to gs://ml_ai_capstone_stock/models/ml_model/fingerprint.pb
Uploaded ml_model/saved_model.pb to gs://ml_ai_capstone_stock/models/ml_model/saved_model.pb
Uploaded ml_model/variables\variables.data-00000-of-00001 to gs://ml_ai_capstone_stock/models/ml_model/variables/variables.data-00000-of-00001
Uploaded ml_model/variables\variables.index to gs://ml_ai_capstone_stock/models/ml_model/variables/variables.index


# Create Endpoint

In [77]:
from google.cloud import aiplatform

aiplatform.init(project="capstone-ml-451002", location="us-central1")

model = aiplatform.Model.upload(
    display_name="stock_prediction_model",
    artifact_uri=f"gs://ml_ai_capstone_stock/models/ml_model/",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest",
)

endpoint = model.deploy(machine_type="n1-standard-4")
print(f"Model deployed at: {endpoint.resource_name}")

Creating Model
Create Model backing LRO: projects/1054820045782/locations/us-central1/models/9046092279550312448/operations/4505490684291055616
Model created. Resource name: projects/1054820045782/locations/us-central1/models/9046092279550312448@1
To use this Model in another session:
model = aiplatform.Model('projects/1054820045782/locations/us-central1/models/9046092279550312448@1')
Creating Endpoint
Create Endpoint backing LRO: projects/1054820045782/locations/us-central1/endpoints/892267979591385088/operations/1588284025661816832
Endpoint created. Resource name: projects/1054820045782/locations/us-central1/endpoints/892267979591385088
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1054820045782/locations/us-central1/endpoints/892267979591385088')
Deploying model to Endpoint : projects/1054820045782/locations/us-central1/endpoints/892267979591385088
Deploy Endpoint model backing LRO: projects/1054820045782/locations/us-central1/endpoints/8922679795

# Convert to Actual Value (Unnormalize)

In [99]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import pandas as pd
Google_stock = pd.read_csv("GOOG.csv")
Google_stock['date'] = pd.to_datetime(Google_stock['date'])
Google_stock['date'] = Google_stock['date'].dt.strftime('%d-%m-%Y')
Google_stock['date'] = pd.to_datetime(Google_stock['date'], dayfirst=True)
## Remove symbol column
Google_stock.drop(columns="symbol",axis=1,inplace=True)
## Feature selection and set date as index
df = Google_stock.loc[:,["date","close","high","low","open","volume"]]
df=df.set_index("date")
# choose close as relevant attributes
train_set = df.iloc[:,:4]
# standardize using Min-Max Scaler
Scaler = MinMaxScaler()
Scaler.fit_transform(train_set)

# Validating Results

In [118]:
# Generate test sequence 
test_set = np.random.rand(10,60,4)  # 60 time steps, 4 features each

# To verify the result from the endpoint, first validate the result from the saved model
reloaded_artifact = tf.saved_model.load("ml_model")
predictions = reloaded_artifact.serve(test_set)
#print(predictions)

# Unnormalize the predictions
unnormalized_predictions = Scaler.inverse_transform(predictions)
unnormalized_predictions = pd.DataFrame(unnormalized_predictions,columns=['close','high','low','open'])

# Print the unnormalized values
print(unnormalized_predictions)

         close         high          low         open
0  1546.435520  1550.478605  1516.304328  1534.378556
1  1521.698605  1547.123358  1486.754352  1523.690448
2  1354.673638  1375.727700  1309.667069  1334.545637
3  1385.304004  1378.875393  1338.506873  1368.291365
4  1537.969223  1571.047744  1506.246597  1547.517605
5  1433.893486  1431.586182  1420.518347  1435.479712
6  1591.770127  1632.987977  1562.037794  1612.180755
7  1561.386326  1618.044743  1530.531594  1567.473205
8  1554.418025  1603.027331  1528.139401  1564.189631
9  1244.818399  1277.114173  1213.520807  1238.310504


# Send Request

In [120]:
import requests
import json
import numpy as np
import google.auth
from google.auth.transport.requests import Request
from google.oauth2 import service_account

# Path to your downloaded service account JSON key
SERVICE_ACCOUNT_FILE = "capstone-ml-451002-bef2906dcf92.json"

# Authenticate and get an OAuth 2 token
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
auth_request = Request()
credentials.refresh(auth_request)
access_token = credentials.token
endpoint_url = f"https://us-central1-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict"
#  Column attributes for reference
COLUMNS = ['close','high','low','open']
# Use the testing set defined from the previous cell and convert to list
instance = test_set.tolist()  # 60 time steps, 4 features each
data = {"instances": instance}  # Wrap in an outer list for batch processing
# Set Authorization header with Bearer Token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
# Send the request and handle errors
try:
    response = requests.post(endpoint_url, json=data, headers=headers)
    response.raise_for_status()  # Raises an error for bad status codes
    predictions = np.array(response.json().get("predictions", []))  # Extract predictions
    # Unnormalize the predictions
    unnormalized_predictions = Scaler.inverse_transform(predictions)
    # Print unnormalized predictions
    df = pd.DataFrame(unnormalized_predictions, columns=COLUMNS)
    df = df[COLUMNS]
    print(df)
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    print(response.text if response is not None else "No response received")

         close         high          low         open
0  1546.435506  1550.478605  1516.304383  1534.378557
1  1521.698604  1547.123469  1486.754408  1523.690448
2  1354.673694  1375.727755  1309.667069  1334.545638
3  1385.304115  1378.875448  1338.506873  1368.291420
4  1537.969278  1571.047854  1506.246597  1547.517716
5  1433.893541  1431.586237  1420.518347  1435.479711
6  1591.770128  1632.987977  1562.037740  1612.180755
7  1561.386399  1618.044853  1530.531538  1567.473205
8  1554.418080  1603.027441  1528.139456  1564.189740
9  1244.818398  1277.114062  1213.520807  1238.310449
